In [71]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='arcticecosystems')

In [72]:
startDate = "2020-06-01"
advanceDays = 30
dateRange = ee.DateRange(ee.Date(startDate), ee.Date(startDate).advance(advanceDays, 'day'))
geometry = ee.Geometry.Point([-129.9930708040777, 68.59277781573002])

In [73]:
testLandsatImage = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').select(
    ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', 'QA_PIXEL'],
    ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'TIR', 'QA_PIXEL']
).filterBounds(geometry).filterDate(dateRange).filter(ee.Filter.lt('CLOUD_COVER', 10))
print(testLandsatImage)

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.filter",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.map",
                      "arguments": {
                        "baseAlgorithm": {
                          "functionDefinitionValue": {
                            "argumentNames": [
                              "_MAPPING_VAR_0_0"
                            ],
                            "body": {
                              "functionInvocationValue": {
                                "functionName": "Image.select",
                                "argumen

In [74]:
band_name_exp = '(b("NIR") - b("Red")) / (b("NIR") + b("Red"))'
band_name_img = testLandsatImage.first().expression(band_name_exp).rename('NDVI')

In [75]:
band_name_exp = '(-0.2848 * b("Blue")) + (-0.2435 * b("Green")) + (-0.5436 * b("Red")) + (0.7243 * b("NIR")) + (0.0840 * b("SWIR1")) + (-0.1800 * b("SWIR2"))'
band_name_img = testLandsatImage.first().expression(band_name_exp).rename('NDVI')

In [76]:
band_name_exp = '(b("SWIR1") - b("NIR")) / (b("SWIR1") + b("NIR"))'
ndbi = testLandsatImage.first().expression(band_name_exp).rename('ndbi')

In [77]:
band_name_exp = '(b("NIR") - b("SWIR1")) / (b("NIR") + b("SWIR1"))'
ndmi = testLandsatImage.first().expression(band_name_exp).rename('ndmi')

In [78]:
band_name_exp = '(b("Green") - b("SWIR1")) / (b("Green") + b("SWIR1"))'
ndwi = testLandsatImage.first().expression(band_name_exp).rename('ndwi')

In [79]:
dataset = ee.ImageCollection('MODIS/061/MCD64A1').filter(ee.Filter.date('2000-11-01', '2024-11-01'));
burnedArea = dataset.select('BurnDate');

In [83]:
Map = geemap.Map()
Map.centerObject(testLandsatImage, zoom = 8)

Map.addLayer(testLandsatImage, {}, 'landsat')
Map.addLayer(gvi, {'min': -5253.5, 'max': 51.7, 'palette': 'Greens'}, 'gvi')
Map.addLayer(ndvi, {'min': 0, 'max': 0.2, 'palette': 'Greens'}, 'ndvi', False)
Map.addLayer(ndbi, {'min': 0, 'max': 0.2, 'palette': 'Reds'}, 'ndbi')
Map.addLayer(ndbi, {'min': -0.1, 'max': 0, 'palette': 'Blues'}, 'ndmi')
Map.addLayer(ndwi, {'min': -0.2, 'max': -0.1, 'palette': 'Purples'}, 'ndwi')
Map.addLayer(burnedArea, {}, 'modis precomp burned area', False)

Map

Map(center=[68.27931430329704, -132.89380539556925], controls=(WidgetControl(options=['position', 'transparent…